In [ ]:
import pandas as pd
import numpy as np
import json
import requests

In [ ]:
'''
Setup Elastic Search Client:
Important Information: DO NOT DELETE
{
  CloudID: "0f90fe69946d46f4b8209dcc27a9f281:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGU4MjA2YjUwNWRjNjQ4Zjc4ZDI0Yjg5NGZlZjE2NWIxJDgzYjY0ZDg3Yjk1NzQ5NmI4OTEzYTlhNmEwMWVmYmQx",
  Url: "https://e8206b505dc648f78d24b894fef165b1.us-central1.gcp.cloud.es.io:443",
}
'''
from elasticsearch import Elasticsearch

client = Elasticsearch(
  "https://e8206b505dc648f78d24b894fef165b1.us-central1.gcp.cloud.es.io:443/",
  api_key="TjA5ckdJd0JKLUQ3RUJkdXdwY2I6Qk9xR2JTUWlTSGVQaUdGbXoxWXRjUQ=="
)

# API key should have cluster monitor rights
client.info()

In [ ]:
final_df = pd.read_csv("D:/Project-Data/predictions_Final.csv")

In [ ]:
print(final_df.columns)
print(final_df.shape)
print(final_df['predictions'].unique())
#Rename columns..
final_df = final_df.rename(columns={'predictions': 'category_code', 'text': 'article'})
print("Renamed Columns")
print(final_df.columns)

Index(['text', 'label', 'predictions'], dtype='object')
(19323, 3)
[0 1 2 3]
Renamed Columns
Index(['article', 'label', 'category_code'], dtype='object')


In [ ]:
#3: science and tech
# create a list of our conditions
conditions = [
    (final_df['category_code'] == 0),
    (final_df['category_code'] == 1),
    (final_df['category_code'] == 2),
    (final_df['category_code'] == 3)
]

# create a list of the values we want to assign for each condition
values = ['world', 'sports', 'business', 'science_and_technology']

# create a new column and use np.select to assign values to it using our lists as arguments
final_df['category'] = np.select(conditions, values)

# display updated DataFrame
final_df.head()

,article,label,category_code,category
0,Popular discontent stirs political awakening i...,0,0,world
1,League officials r williams s attorney meet ri...,1,1,sports
2,Us airline starts vietnam flights the us resum...,0,0,world
3,Saks posts wider second quarter loss reuters r...,2,2,business
4,Ecb bank of england leave key rates alone the ...,0,2,business


In [ ]:
json_obj = final_df.to_json(orient='records', lines=True)

In [ ]:
json_split = json_obj.split('\n')
print(len(json_split))

19324


In [ ]:
count = 0
json_split = json_obj.split('\n')
documents_500 = []
for i in range(0, len(json_split)):
    try:
        if(len(json_split[i]) == 0):
            continue

        jdict = json.loads(json_split[i])
        metadata = {"index": {"_id": count, "_index": "search-news-article-data-298b"}}
        final_json_string = jdict
        documents_500.append(metadata)
        documents_500.append(final_json_string)

        count += 1
        if len(documents_500) == 500:
            client.bulk(operations=documents_500, pipeline="ent-search-generic-ingestion")
            documents_500 = []
            print(f"Indexed documents: {count}")

    except:
        print(f"Error for {json_split[i]}")

if len(documents_500) > 0:
    client.bulk(operations=documents_500, pipeline="ent-search-generic-ingestion")

print(f"Total Indexed documents: {count}")
# print(documents)

Indexed documents: 250
Indexed documents: 500
Indexed documents: 750
Indexed documents: 1000
Indexed documents: 1250
Indexed documents: 1500
Indexed documents: 1750
Indexed documents: 2000
Indexed documents: 2250
Indexed documents: 2500
Indexed documents: 2750
Indexed documents: 3000
Indexed documents: 3250
Indexed documents: 3500
Indexed documents: 3750
Indexed documents: 4000
Indexed documents: 4250
Indexed documents: 4500
Indexed documents: 4750
Indexed documents: 5000
Indexed documents: 5250
Indexed documents: 5500
Indexed documents: 5750
Indexed documents: 6000
Indexed documents: 6250
Indexed documents: 6500
Indexed documents: 6750
Indexed documents: 7000
Indexed documents: 7250
Indexed documents: 7500
Indexed documents: 7750
Indexed documents: 8000
Indexed documents: 8250
Indexed documents: 8500
Indexed documents: 8750
Indexed documents: 9000
Indexed documents: 9250
Indexed documents: 9500
Indexed documents: 9750
Indexed documents: 10000
Indexed documents: 10250
Indexed documents

In [ ]:
# Run only when required.. Delete all the documents..
# client.delete_by_query(index=['search-news-article-data-298b'], body={"query": {"match_all": {}}})

ObjectApiResponse({'took': 1117, 'timed_out': False, 'total': 25711, 'deleted': 25711, 'batches': 26, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [ ]:
# DO NOT RUN AGAIN
# headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
# r = requests.post('http://localhost:9200/news-articles/_bulk', data=final_json_string, headers=headers, timeout=60)